# Network Visualization

This notebook loads a dataset and network and visualized the dataset features and subsequent network activations

In [ ]:
import os
import tensorflow as tf
from Dataset import Dataset
import matplotlib.pyplot as plt
# custom functions for interactiva plotting
from utils.interactive_plotting import show, show_gray, show_rgb
# custom functions for writing pngs
from utils.interactive_plotting import norm_ptp, norm_std, norm_rgb, write, dump3, dump, dump_rgb, dump_class
import numpy as np
from PIL import Image
%matplotlib inline

tf.reset_default_graph()

modeldir="data_IJGI18/models/convlstm256_48px"

# None -> dont write output pngs
outfolder=None

datadir="data_IJGI18/datasets/demo/480"

## Initialize Dataset

If `RessourceExhaustedError` or `OutOfMemory` error occur later: reduce number ids in `overwrite_ids` and `batchsize`

In [ ]:
# set specificly which ids should be returned (set none for random)
# tile ids from paper: 1823 8133 2550 2554 10791 10792 10879 10969 12894 16494 1272
overwrite_ids=[2550,10969]
batchsize=2

dataset = Dataset(datadir=datadir, 
                  verbose=True, 
                  temporal_samples=None, 
                  section="2016")

tfdataset, _, _, filenames = dataset.create_tf_dataset("eval",0,batchsize,True,2,overwrite_ids=overwrite_ids)
iterator = tfdataset.make_initializable_iterator()

## Start TF Session and initialize iterators

In [ ]:
config = tf.ConfigProto()
sess = tf.InteractiveSession()

data_handle = sess.run(iterator.string_handle())

#train_writer = tf.summary.FileWriter(os.path.join(args.modeldir, TRAINING_SUMMARY_FOLDER_NAME), sess.graph)
#test_writer = tf.summary.FileWriter(os.path.join(args.modeldir, TESTING_SUMMARY_FOLDER_NAME))

sess.run([iterator.initializer])

## Checkout input data from the iterator

In [ ]:
sess.run([tf.global_variables_initializer(), tf.local_variables_initializer(), tf.tables_initializer()])

# run a network session with the operations defined prviously
x10,x20,x60,doy,year,labels = sess.run(iterator.get_next())

In [ ]:
show_rgb(x10)
show_rgb(x20)
show_rgb(x60)
show_gray(labels[:,0,:,:],name="labels")

## load model graph definition (graph.meta)

In [ ]:
graph=os.path.join(modeldir,"graph.meta")
tf.train.import_meta_graph(graph)

# initialize variables
sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])

## restore weights from checkpoint (model.ckpt)

In [ ]:
saver = tf.train.Saver(save_relative_paths=True)
checkpoint = os.path.join(modeldir, "model.ckpt")

latest_ckpt = tf.train.latest_checkpoint(modeldir)
if latest_ckpt is not None:
    print "restoring from " + latest_ckpt
    saver.restore(sess, latest_ckpt)

## List of all available operations within the model graph

In [ ]:
sess.graph.get_operations()
# tf.trainable_variables()

## Retrieve operations from the graph

In [ ]:
def get_op(name):
    return tf.get_default_graph().get_operation_by_name(name).outputs[0]

## get variables from tf.default_graph
iterator_handle_op = get_op("data_iterator_handle")
is_train_op = get_op("is_train")
global_step_op = get_op("global_step")
train_op = get_op("train_op")

# container to store operations to be obtained
query_map=dict()

#atrousdeep generation
query_map["x"]="input/reshaped/x"

query_map["convrnn_input"]="convrnn1/input"
query_map["convrnn_output"]="convrnn1/outputs"
query_map["convrnn_state"]="convrnn1/final_states"
    
query_map["targets"]="targets"
query_map["predictions"]="predictions"
query_map["prediction_scores"]="prediction_scores"
query_map["correctly_predicted"]="correctly_predicted"

## Inference - Execute the graph with data and store results in dict

In [ ]:
feed = {iterator_handle_op:data_handle,is_train_op:False}

operations = ops = [get_op(query_map[key]) for key in sorted(query_map.keys())]

# run a network session with the operations defined prviously
queried = sess.run(operations,feed_dict=feed)

# build a results dict
results=dict()
for key, array in zip(sorted(query_map.keys()),queried):
    results[key]=array

In [ ]:
print(results.keys())

## Visualize Results

### Input

In [ ]:
show_rgb(results["x"], name="x")
show_gray(results["targets"],name="labels")

## Sequence encoder input and output

In [ ]:
show(results["convrnn_input"],name="convrnn_in")
show(results["convrnn_state"],name="states")

if outfolder is not None:
    dump3(array=results["convrnn_state"], name="final_state", outfolder=outfolder, cmap="inferno")
    dump3(array=results["prediction_scores"], name="prediction_scores", outfolder=outfolder, cmap="inferno")
#dump(array=results["convrnn_state"], name="final_state", outfolder=outfolder, cmap="inferno")

### Activations for each class (aka band)

In [ ]:
show_gray(results["prediction_scores"],name="prediction scores")

### Predictions vs Ground truth

In [ ]:
show_gray(results["predictions"],name="predictions",vmin=0,vmax=26)
show_gray(results["targets"],name="labels",vmin=0,vmax=26)

## Recreate LSTM iteration to access gate activations

also implemented in activations.py

try:
```
python activations.py --help
```
for more info

### Define the LSTM cell
see implementation in ```utils/convrnn/cell.py```

In [ ]:
import utils.convrnn as convrnn
import numpy as np

b,t,px,px,d_in = results["convrnn_input"].shape

def convolution(inputs,W,data_format):
    """wrapper around tf.nn.convolution with custom padding"""
    pad_h = int(W.get_shape()[0])/2
    pad_w = int(W.get_shape()[1])/2

    paddings = tf.constant([[0, 0], [pad_h,pad_h], [pad_w,pad_w], [0, 0]])

    inputs_padded = tf.pad(inputs, paddings, "REFLECT")

    return tf.nn.convolution(inputs_padded, W, 'VALID', data_format=data_format)

def layer_norm(inputs,beta,gamma):
    """taken from contrib tf.contrib.layers.layer_norm definition in
    tensorflow/contrib/layers/python/layers/layers.py
    """
    mean, variance = tf.nn.moments(inputs, [1,2,3], keep_dims=True)
    outputs = tf.nn.batch_normalization(
        inputs, mean, variance, offset=beta, scale=gamma,
        variance_epsilon=1e-12)
    return outputs

x = results["convrnn_input"][:,0]

scope = "convrnn1/bidirectional_rnn/fw/conv_lstm_cell"


#state = sess.run(zero_state_op)

weights=[]
weights.append(get_op(scope+"/kernel"))
#weights.append(get_op(scope+"/W_ci")) # peephole
#weights.append(get_op(scope+"/W_cf")) # peephole
#weights.append(get_op(scope+"/W_co")) # peephole
weights.append(get_op(scope+"/LayerNorm/beta"))
weights.append(get_op(scope+"/LayerNorm/gamma"))
weights.append(get_op(scope+"/LayerNorm_1/beta"))
weights.append(get_op(scope+"/LayerNorm_1/gamma"))
weights.append(get_op(scope+"/LayerNorm_2/beta"))
weights.append(get_op(scope+"/LayerNorm_2/gamma"))
weights.append(get_op(scope+"/LayerNorm_3/beta"))
weights.append(get_op(scope+"/LayerNorm_3/gamma"))
weights.append(get_op(scope+"/LayerNorm_4/beta"))
weights.append(get_op(scope+"/LayerNorm_4/gamma"))

def lstm(x,state,weights, peephole=False, activation=tf.nn.tanh):
    """Implementation modified from carlthome/tensorflow-convlstm-cell"""

    if peephole:
        kernel, W_ci, W_cf, W_co, b_j, g_j, b_i, g_i, b_f, g_f, b_o, g_o, b_c, g_c = weights
    if not peephole:
        kernel, b_j, g_j, b_i, g_i, b_f, g_f, b_o, g_o, b_c, g_c = weights
    
    
    c,h = state
    x = tf.concat([x, h], axis=3).eval()
    n = x.shape[-1]
    m = 4 * convfilters if convfilters > 1 else 4
    y = convolution(x,kernel,data_format="NHWC").eval()
    #y = tf.nn.convolution(x, kernel, 'SAME', data_format="NHWC").eval()
    j, i, f, o = tf.split(y, 4, axis=3)

    if peephole:
        # peephole connections
        i += W_ci * c
        f += W_cf * c

    # normalize
    # replacement for tf.contrib.layers.layer_norm(j)
    #
    ## normalize in cell.py
    #j = tf.contrib.layers.layer_norm(j)
    #i = tf.contrib.layers.layer_norm(i)
    #f = tf.contrib.layers.layer_norm(f)
    #
    j = layer_norm(j,b_j,g_j)

    i = layer_norm(i,b_i,g_i)

    f = layer_norm(f,b_f,g_f)

    forget_bias=1
    f = tf.sigmoid(f + forget_bias)
    i = tf.sigmoid(i)
    c = c * f + i * activation(j)
    
    if peephole:
        o += W_co * c

    o = layer_norm(o,b_o,g_o)

    c = layer_norm(c,b_c,g_c)

    o = tf.sigmoid(o)
    h = o * activation(c)

    state = tf.nn.rnn_cell.LSTMStateTuple(c, h)

    return h,state,j,i,f,o


In [ ]:
results["convrnn_output"].shape

### Perform Iteration on CPU

In [ ]:
inputs = results["convrnn_input"]

b,t,px,px,d = results["convrnn_output"].shape
convfilters = d/2
zero_state_op = tf.contrib.rnn.LSTMStateTuple(c=tf.zeros(tf.TensorShape([b, px, px, convfilters])),h=tf.zeros(tf.TensorShape([b, px, px, convfilters])))
state=sess.run(zero_state_op)

jGate = []
iGate = []
fGate = []
oGate = []
outputs = []
states = []
statesh = []

# execute on cpu because ressource exhausted error on GPU
with tf.device('/cpu:0'):

    for time in range(0,t):
        print("time {}".format(time))

        h,state,j,i,f,o = lstm(inputs[:,time],state, weights)
        state = tf.contrib.rnn.LSTMStateTuple(c=state.c.eval(),h=state.h.eval())

        #show_gray(i.eval(),"input_gate at t{}".format(it))

        iGate.append(i.eval())
        jGate.append(j.eval())
        fGate.append(f.eval())
        oGate.append(o.eval())
        outputs.append(h.eval())
        states.append(state.c)
        statesh.append(state.h)

iGate = np.stack(iGate,axis=1)
jGate = np.stack(jGate,axis=1)
fGate = np.stack(fGate,axis=1)
oGate = np.stack(oGate,axis=1)
outputs = np.stack(outputs,axis=1)
states = np.stack(np.array(states),axis=1)
statesh = np.stack(np.array(statesh),axis=1)

### Show Results

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

def show_activations(arrays_tuple, titles=[""]):
    
    max_b, max_t,_,_,max_d = arrays_tuple[1].shape
    max_d=10
        
    def _show_map_BTHWD(t,d,b): 
        n = len(arrays_tuple)
        
        fig,axs = plt.subplots(1,n,figsize=(16,6))
        
        for array,ax,title in zip(arrays_tuple,axs,titles): 
            ax.imshow(array[b,t,:,:,d-1:d+2])
            ax.set_title(title)
            #_show(array[b,t,:,:,d],"{name} feature map b={b}, t={t}, d={d}".format(name=name,b=b,t=t,d=d))
    
    # all
    b_slider = widgets.IntSlider(description='batch',min=0,max=max_b-1,step=1,value=max_b/2)
         
    d_slider = widgets.IntSlider(description='band',min=1,max=max_d-1,step=1,value=max_d/2) 
    t_slider = widgets.IntSlider(description='time',min=0,max=max_t-1,step=1,value=max_t/2)
    w = interactive(_show_map_BTHWD, t=t_slider, d=d_slider, b=b_slider)
    
    w.layout.height = '400px'
    display(w)
   
#show(,"x")
def norm(arr,thresmin=-1,thresmax=1):
    arr[arr<thresmin]=thresmin
    arr[arr>thresmax]=thresmax
    return ( (arr-arr.min()) / (arr-arr.min()).max()).astype('float')

show_activations([iGate,oGate,norm(jGate),fGate,norm(states),norm(statesh),norm(outputs)],titles=["i","o","j","f","state","stateh","output"])
show(results["x"])
show_gray(states)
show_gray(statesh)

In [ ]:
show_rgb(results["x"], name="x")
show_gray(iGate,"input")
show_gray(fGate,"forget")
show_gray(states,"states")

## Write Arrays to png files

folder structure
``
{outfolder}/sample{s}/time{t}/{d}_{name}.png
``

with `s` as sample in batch, `t` as time, `d` as number of feature map and `name` `iGate`,`jGate`,`oGate`, etc.

In [ ]:
if outfolder is not None:

    cmap="inferno"

    dump_rgb(results["x"][:,:,:,:,0:3],"x",outfolder,stddev=4)

    dump(array=iGate, name="iGate", outfolder=outfolder, cmap="inferno")
    dump(array=fGate, name="fGate", outfolder=outfolder, cmap="inferno")
    dump(array=oGate, name="oGate", outfolder=outfolder, cmap="inferno")
    dump(array=(jGate/2)+0.5, name="jGate", outfolder=outfolder, cmap="RdBu_r")
    dump(array=(statesh/2)+0.5, name="output", outfolder=outfolder, cmap="RdBu_r")
    dump(array=(states/2)+0.5, name="state", outfolder=outfolder, cmap="RdBu_r")

    dump_class(results["targets"],"ground_truth",outfolder)
    dump_class(results["predictions"],"predictions",outfolder)
    for i in range(0,17):
        dump_class(results["prediction_scores"][:,:,:,i],"prediction_scores_"+str(i),outfolder)

In [ ]:
results["prediction_scores"].shape

In [ ]:
def plot_color_gradients(name, filename="tmp/test.png",w=8,h=2):
    fig = plt.figure(frameon=False)
    fig.set_size_inches(w,h)
    #fig.subplots_adjust(top=0.95, bottom=0.01, left=0.2, right=0.99)

    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    ax.imshow(gradient, aspect='auto', cmap=plt.get_cmap(name))

    ax.set_axis_off()
        
    plt.savefig(filename)

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

if not os.path.exists("tmp"):
    os.makedirs("tmp")

plot_color_gradients("inferno", w=12,h=3,filename=os.path.join("tmp","inferno.png"))
plot_color_gradients("RdBu", w=12,h=3,filename=os.path.join("tmp","RdBu.png"))
plot_color_gradients("RdBu_r", w=12,h=3,filename=os.path.join("tmp","RdBu_r.png"))